# Nested Cross-Validation

*Nested Cross-Validation* algorithm built from scratch under the form of the `nested_cv()` Python function.

In [ ]:
def nested_cv(features, labels, f_scaling, grid_alpha, k_inner, k_outer, seed, include_intercept = True):
    
    # Arrays for tracking outer Training, Validation errors e the best Hyperparametrs tuned in the inner loop
    outval_errors = np.zeros((1, k_outer))
    innval_errors = np.zeros((len(grid_alpha), k_inner))
    bestinn_alphas = np.zeros((1, k_outer))
    
    # Random shuffle of the Features and Labels
    idx_nested = np.random.RandomState(seed = seed).permutation(len(features))
    feat_shuff, labl_shuff = features.loc[idx_nested, :], labels.loc[idx_nested, :]
    
    # Features and Labels divided in "k_outer" number of folds
    feat_out_folds, labl_out_folds = np.array_split(feat_shuff, k_outer), np.array_split(labl_shuff, k_outer)
    
    # Loop on the outer Training and Validation Folds
    for i in range(0, k_outer):
        
        # Features and Labels in the outer Validation fold at each iteration
        outf_val = feat_out_folds[i]
        outl_val = labl_out_folds[i]
        
        # Features and Labels in the outer Training folds at each iteration
        outf_train = feat_shuff.drop(outf_val.index)
        outl_train = labl_shuff.drop(outl_val.index)
        
        # Training Features and Labels divided in "k_inner" number of folds
        feat_inn_folds, labl_inn_folds = np.array_split(outf_train, k_inner), np.array_split(outl_train, k_inner)
        
        # Loop on the values of the Hyperparameter in the grid 
        for alp in range(0, len(grid_alpha)):
            
            # Loop on the inner Training and Validation folds
            for inn in range(0, k_inner):
                
                # Features and Labels in the inner Validation fold at each iteration
                innf_val = feat_inn_folds[inn]
                innl_val = labl_inn_folds[inn]
                
                # Features and Labels in the inner Training folds at each iteration
                innf_train = outf_train.drop(innf_val.index)
                innl_train = outl_train.drop(innl_val.index)
                
                # Transform the Features without breaking the independence between Training and Validation inner folds
                if f_scaling is not None:
                    transformer = features_transformation(scale_transform = f_scaling)
                    innf_train = transformer.fit(innf_train)
                    innf_val = transformer.test_transform(innf_val)
                
                # Train the predictor on the inner Training folds
                ridge_nested = Ridge(alpha = grid_alpha[alp], intercept = include_intercept)
                ridge_nested.fit(innf_train, innl_train)     
                
                # Compute the inner Validation Error according to the square loss
                innval_pred = ridge_nested.predict(innf_val)
                innval_errors[alp, inn] = np.mean((innval_pred - innl_val)**2)
        
        # Retrieve the best Hyperparameter from the inner loop according to the corresponding inner Validation Error
        innval_mean = np.mean(innval_errors, axis = 1)
        bestinn_alphas[0,i] = grid_alpha[np.where(innval_mean == np.min(innval_mean))]
        
        # Transform the Features without breaking the independence between Training and Validation outer folds
        if f_scaling is not None:
            transformer = features_transformation(scale_transform = f_scaling)
            outf_train = transformer.fit(outf_train)
            outf_val = transformer.test_transform(outf_val)
        
        # Train the predictor with the best Hyperparameter on the outer Training folds
        ridge_outer = Ridge(alpha = bestinn_alphas[0,i], intercept = include_intercept)
        ridge_outer.fit(outf_train, outl_train)
        
        # Compute the outer Validation Error according to the square loss        
        outval_pred = ridge_outer.predict(outf_val)
        outval_errors[0, i] = np.mean((outval_pred - outl_val)**2)
            
    # Compute the average of the Outer Validation Errors as an estimate for the risk
    nestedcv_score = np.mean(outval_errors)
    print("Nested CV risk estimate is: ")
        
    return(np.format_float_scientific(nestedcv_score))